In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

import folium
from folium.plugins import HeatMap

import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster


In [ ]:
# read file
df = pd.read_csv('../input/tsunami-dataset/tsunami_dataset.csv')
df

In [ ]:
df.info()

In [ ]:
# check dimensions
df.shape

In [ ]:
# show all column names
df.columns

In [ ]:
zoom_factor = 1.1 # setting for maps

In [ ]:
#TSUNAMI INTENSITY
plt.figure(figsize=(15,7))
plt.scatter(df.YEAR, df.TS_INTENSITY, c='#00006040')
plt.xlabel('Year')
plt.ylabel('TSUNAMI_INTENSITY')
plt.xlim(1900,2020)
plt.grid()
plt.show()

In [ ]:
#EQ_MAGNITUDE
plt.figure(figsize=(15,7))
plt.scatter(df.YEAR, df.EQ_MAGNITUDE, c='#00006040')
plt.xlabel('Year')
plt.ylabel('MAGNITUDE')
plt.xlim(1900,2020)
plt.grid()
plt.show()

In [ ]:
#EQ_DEPTH
plt.figure(figsize=(15,7))
plt.scatter(df.YEAR, df.EQ_DEPTH, c='#00006040')
plt.xlabel('Year')
plt.ylabel('EQ_DEPTH')
plt.xlim(1900,2020)
plt.grid()
plt.show()

In [ ]:
# filter by magnitude
df_mg = df.loc[df.EQ_MAGNITUDE >5].copy()

In [ ]:
# plot
plt.figure(figsize=(16,4))
plt.scatter(df_mg.YEAR, df_mg.EQ_MAGNITUDE)
plt.xlabel('YEAR')
plt.ylabel('EQ_MAGNITUDE')
plt.title('EQ MAGNITUDE > 5')
plt.xlim(1900,2020)
plt.grid()
plt.show()

In [ ]:
#plot 
plt.figure(figsize=(16,4))
df.CAUSE.value_counts()[0:10].plot(kind='bar')
plt.title('cause of the tsunami')
plt.grid()
plt.show()

In [ ]:
import plotly.graph_objects as go
#matplotlib inline
import squarify
import matplotlib
import matplotlib as mpl
import matplotlib.cm as cm

In [ ]:
fig = go.Figure(go.Bar(y=df['CAUSE'],
                       x=df.ID ,
                       orientation='h',
        marker={
        'color': [np.random.randint(10,255) for x in range(0,len(df))],
        'colorscale': 'Viridis'
        }
        ))

fig.update_layout(yaxis=dict(title='Cause'),width=900,height=500,title='Cause of the tsunami',
                 xaxis=dict(title='Value'))
fig.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 35})

plt.figure(figsize=(40,15))
CAUSE_count = df['CAUSE'].value_counts()
squarify.plot(sizes= CAUSE_count.values,label=CAUSE_count.index,alpha=0.7)

In [ ]:
# plot Fire Spot vs Year
plt.figure(figsize=(10,8))
plt.bar(df.YEAR, df.EVENT_VALIDITY , color='blue', alpha=0.25)
plt.xlabel('Year')
plt.ylabel('Event Validity')
plt.xlim(1900,2020)
plt.grid()
plt.show()

In [ ]:
# convert lat/lon to numeric
df_mg.loc[:,'LATITUDE'] = pd.to_numeric(df_mg['LATITUDE'], downcast='float')
df_mg.loc[:,'LONGITUDE'] = pd.to_numeric(df_mg['LONGITUDE'], downcast='float')

In [ ]:
sel_columns = ['YEAR','EQ_MAGNITUDE','LATITUDE','LONGITUDE']
df_mg = df_mg[sel_columns]
df_mg

In [ ]:
#Marker Maps 
my_map_1 = folium.Map(location=[0,0], zoom_start=zoom_factor)
#Add points to the map
for idx, row in df_mg.iterrows():
    Marker([row['LATITUDE'], row['LONGITUDE']]).add_to(my_map_1)


my_map_1 # display

In [ ]:
#MarkerCluster Maps

# Create the map
my_map_2 = folium.Map(location=[0,0], zoom_start=zoom_factor)

# Add points to the map
mc = MarkerCluster()
for idx, row in df_mg.iterrows():
    if not math.isnan(row['LONGITUDE']) and not math.isnan(row['LATITUDE']):
        mc.add_child(Marker([row['LATITUDE'], row['LONGITUDE']]))
my_map_2.add_child(mc)

# Display the map
my_map_2

In [ ]:


#Bubble maps
# Create a base map
my_map_3 = folium.Map(location=[0,0], zoom_start=zoom_factor)

def color_producer(val):
    if val <= 7:
        return 'forestgreen'
    else:
        return 'darkred'

# Add a bubble map to the base map
for i in range(0,len(df_mg)):
    Circle(
        location=[df_mg.iloc[i]['LATITUDE'], df_mg.iloc[i]['LONGITUDE']],
        radius=20,
        color=color_producer(df_mg.iloc[i]['EQ_MAGNITUDE'])).add_to(my_map_3)

# Display the map
my_map_3

In [ ]:
#Heatmaps
# Create a base map
my_map_4 = folium.Map(location=[0,0], zoom_start=zoom_factor)

# Add a heatmap to the base map
HeatMap(data=df_mg[['LATITUDE', 'LONGITUDE']], radius=10).add_to(my_map_4)

# Display the map
my_map_4

In [ ]:
#Combine heat map and Bubble maps
my_map_5 = my_map_3

HeatMap(data=df_mg[['LATITUDE', 'LONGITUDE']], radius=10).add_to(my_map_5)

my_map_5